In [8]:
%load_ext autoreload
%autoreload 2

import argparse

# import pytorch_lightning as pl
import torch
import torch.utils.data as data
import wandb
# from pytorch_lightning.loggers import WandbLogger

from dataset import process_dataset
from model import get_model_tok
# from utils import config_from_args
import numpy as np
from datasets import load_dataset
from evaluate import load
import evaluate

import os
from metric.metrics import compute_metrics
import nltk
nltk.download('punkt')


# rouge = evaluate.load("rouge")

import numpy as np

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

[nltk_data] Downloading package punkt to /home/erencan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# Function for setting the seed
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
set_seed(42)


# from sklearn.model_selection import train_test_split

# dataset = XSUM()
# dataset = load_dataset("xsum",  split="train[:1%]")
dataset = load_dataset("xsum",  split="validation[:1%]")

# model, tokenizer = get_model_tok(cfg.model)
model, tokenizer = get_model_tok('t5-small')
# model, tokenizer = get_model_tok('google/t5-efficient-tiny')

train_dataset, eval_dataset, data_collator = process_dataset(dataset, tokenizer, model)
# tokenized_dataset, data_collator = process_dataset(dataset, tokenizer, model)

# using datetime module YYYY_MM_DD_HH_MM
from datetime import datetime
ID = datetime.now().strftime("%Y_%m_%d_%H_%M")
model_id = f"models/testing_{ID}/"

# torch.cuda.set_device(1)
print(torch.cuda.current_device())

# print available devices
print("Available devices:")
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_id}/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,  # TODO Understand Param.
    num_train_epochs=3,
    predict_with_generate=True,
    # fp16=True, only if cuda
    # push_to_hub=True,
    # report_to="wandb",
)


trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,  # Use the same data collator for both train and eval
        # compute_metrics=lambda pred: compute_metrics(pred, tokenizer, eval_dataset),
        # compute_metrics=compute_metrics,  # Pass the compute_metrics function directly

        # logger="wandb",
        # device=cfg.device,
    )

model t5-small
Using prefix summarize: 
Size of train set 90
Size of eval set 23


Map: 100%|██████████| 23/23 [00:00<00:00, 239.45 examples/s]

MAX_LEN 60
0
Available devices:
1
NVIDIA GeForce MX130


In [3]:
for i in eval_dataset:
    print(i.keys())
    break

dict_keys(['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'])


In [10]:
torch.cuda.empty_cache()

from metric.metrics import *

factsumm = FactSumm()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for _ in range(1):
    eval_results = trainer.predict(eval_dataset) # .evaluate() --> predict + compute metrics

    # Extract relevant information for custom metric computation
    # (batch, DIM)
    document_texts = [example['document'] for example in eval_dataset]
    summary_texts = [example['summary'] for example in eval_dataset]
    summary_texts_pred = eval_results.predictions

    print(document_texts)

    print("-------------------")
    print(summary_texts_pred)
    # custom_metric = compute_metrics(tokenizer, document_texts, summary_texts, summary_texts_pred, device)
    

    # document_texts, summary_texts, summary_texts_pred = document_texts, summary_texts, summary_texts_pred
    # num_samples = len(document_texts)
    # ensemble_scores = []

    # factsumm = FactSumm()

    # for i in range(num_samples):
    #     article = document_texts[i]
    #     summary_texts = summary_texts[i]  # List of vocab IDs
    #     summary_pred_ids = summary_texts_pred[i]  # List of vocab IDs
        
    #     # PREDICTED
    #     # Decode the summary_ids and summary_pred_ids to text summaries
    #     print("summary_pred_ids", summary_pred_ids)
    #     summary_pred_text = tokenizer.decode(summary_pred_ids, skip_special_tokens=True)
    #     print("summary_pred_text", summary_pred_text)
    #     summary_pred_text = summary_pred_text if type(summary_pred_text) == list else summary_pred_text
        
    #     article, summary_texts = str(article), str(summary_texts)
    #     print(f"article {type(article)}")
    #     print(f"summary_texts {type(summary_texts)}")
    #     print("summary", type(summary_pred_text))
    #     summary = summary_pred_text
        
    #     print("device", device)

    #     print("article", article)
    #     print("summary", summary)

    #     BERT = factsumm.calculate_bert_score(article, summary)
    #     print("BERT", BERT)
    #     torch.cuda.empty_cache()

        # from bleurt import score
        # import os
        # # check in model for BLEURT-20-D3 or BLEURT-20    
        # if os.path.exists("metric/bleurt/bleurt/BLEURT-20-D3/"):
        #     checkpoint = "metric/bleurt/bleurt/BLEURT-20-D3/"
        # else:
        #     checkpoint = "metric/bleurt/bleurt/BLEURT-20/"
        # scorer = score.BleurtScorer(checkpoint)
        # BLEURT = scorer.score(references=[article], candidates=[summary])
        # print("BLEURT", BLEURT)
        # torch.cuda.empty_cache()

     
    # break 
# print(f"Epoch {epoch + 1} - Custom Metric ensemble: {custom_metric['ensemble']}")



['Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.\nShe said there "probably was an element" of Labour\'s Remain campaign starting too late.\n"A lot of us were exhausted", she told BBC Radio Wales.\nSenior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.\nPlaid leader no longer ruling out coalition with Labour\nAMs to have say on Brexit deal, says Welsh Tory leader\nCorbyn\'s Labour set for heavy losses, says Stephen Kinnock\nAsked if Labour\'s Remain campaign started too late, she told the Eye on Wales programme: "I think there probably was an element of that.\n"And don\'t forget in Wales we had the assembly elections where people were not talking about the European Union. We were focused absolutely on those assembly elections."\nShe added: "I think it was a mistake not to be talking about the European referendum 

In [13]:
a = document_texts[0]
a = a.replace("\n", "\n")
summary_pred_text = tokenizer.decode(summary_texts_pred[0], skip_special_tokens=True)
b = summary_pred_text

In [18]:
print(a)

Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.
She said there "probably was an element" of Labour's Remain campaign starting too late.
"A lot of us were exhausted", she told BBC Radio Wales.
Senior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.
Plaid leader no longer ruling out coalition with Labour
AMs to have say on Brexit deal, says Welsh Tory leader
Corbyn's Labour set for heavy losses, says Stephen Kinnock
Asked if Labour's Remain campaign started too late, she told the Eye on Wales programme: "I think there probably was an element of that.
"And don't forget in Wales we had the assembly elections where people were not talking about the European Union. We were focused absolutely on those assembly elections."
She added: "I think it was a mistake not to be talking about the European referendum in those assemb

In [14]:
print(type(a), type(b))

<class 'str'> <class 'str'>


In [15]:
factsumm.calculate_bert_score(b,a)

Loading BERTScore Pipeline...

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [2, 768] but got: [1, 768].

In [4]:
article
# article = article.replace('"',"'")
article = article.replace("'", '"')
article

'Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.\nShe said there "probably was an element" of Labour"s Remain campaign starting too late.\n"A lot of us were exhausted", she told BBC Radio Wales.\nSenior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.\nPlaid leader no longer ruling out coalition with Labour\nAMs to have say on Brexit deal, says Welsh Tory leader\nCorbyn"s Labour set for heavy losses, says Stephen Kinnock\nAsked if Labour"s Remain campaign started too late, she told the Eye on Wales programme: "I think there probably was an element of that.\n"And don"t forget in Wales we had the assembly elections where people were not talking about the European Union. We were focused absolutely on those assembly elections."\nShe added: "I think it was a mistake not to be talking about the European referendum in th

In [6]:
print(summary)
summary = summary.replace("'", '"')
summary

baroness Morgan says there "probably was an element" of Labour"s Remain campaign


'baroness Morgan says there "probably was an element" of Labour"s Remain campaign'

In [14]:
a = """Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.\nShe said there "probably was an element" of Labour"s Remain campaign starting too late.\n"A lot of us were exhausted", she told BBC Radio Wales.\nSenior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.\nPlaid leader no longer ruling out coalition with Labour\nAMs to have say on Brexit deal, says Welsh Tory leader\nCorbyn"s Labour set for heavy losses, says Stephen Kinnock\nAsked if Labour"s Remain campaign started too late, she told the Eye on Wales programme: "I think there probably was an element of that.\n"And don"t forget in Wales we had the assembly elections where people were not talking about the European Union. We were focused absolutely on those assembly elections."\nShe added: "I think it was a mistake not to be talking about the European referendum in those assembly elections."\nWales voted 52.5% to 47.5% for Brexit. Only five counting areas voted to Remain.\nThe strongest Leave votes were in Torfaen (60%) and Blaenau Gwent (62%) - two traditional Labour strongholds in the south Wales valleys.\nThe Welsh government has a programme to "re-energise" communities that voted to Leave, much of which would have been funded by EU money, Baroness Morgan said.\n"The question for us now is how on Earth do we reorganise the economy without that European funding?" she asked.\n"Unless we find a way to reconnect with that working class element within our communities then I think the Labour party is in a serious situation in future".\nLabour won 29 seats at May"s assembly election - a better result than many expected - but lost 7.6% of its share of the vote on the constituency ballot.\nEarlier, on BBC Sunday Politics Wales, Baroness Morgan said Carwyn Jones was "absolutely, without question" the right man to lead Welsh Labour."""
a = a.replace('\n', ' ')
a = a.replace("'", '"')
b = """baroness Morgan says there 'probably was an element" of Labour's Remain campaign"""
# BERT = factsumm.calculate_bert_score(article, summary)
BERT = factsumm.calculate_bert_score(b,a)


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [2, 768] but got: [1, 768].

In [20]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
path = "metric/bleurt/bleurt/bleurt/BLEURT-20-D3"
bleurt = load("bleurt", module_type="metric", checkpoint=path)
results = bleurt.compute(predictions=predictions, references=references)

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: evaluate.load('bleurt', 'bleurt-large-512').


AttributeError: 'DownloadConfig' object has no attribute 'token'

In [2]:
from bleurt import score

checkpoint = "metric/BLEURT-20-D3"
references = ["This is a test."]
candidates = ["This is the test."]

scorer = score.BleurtScorer(checkpoint)
scores = scorer.score(references=references, candidates=candidates)
assert isinstance(scores, list) and len(scores) == 1
print(scores)

INFO:tensorflow:Reading checkpoint metric/BLEURT-20-D3.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20-D3
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20-D3
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: metric/BLEURT-20-D3/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2023-10-03 16:54:20.054408: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-03 16:54:20.058274: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-03 16:54:20.058763: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.
2023-10-03 16:54:23.838353: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:521] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.8
  /usr/local/cuda
  /home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.
2023-10-03 16:54:23.861106: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file 

UnknownError: Graph execution error:

Detected at node bert/embeddings/LayerNorm/batchnorm/Rsqrt defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/site-packages/traitlets/config/application.py", line 1046, in launch_instance

  File "/home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/erencan/anaconda3/envs/DL4NLP

In [10]:

article1 = "Lionel Andrés Messi (born 24 June 1987) is an Argentine professional footballer who plays as a forward and captains both Spanish club Barcelona and the Argentina national team. Often considered as the best player in the world and widely regarded as one of the greatest players of all time, Messi has won a record six Ballon d'Or awards, a record six European Golden Shoes, and in 2020 was named to the Ballon d'Or Dream Team."
summary1 = "Lionel Andrés Messi (born 24 Aug 1997) is an Spanish professional footballer who plays as a forward and captains both Spanish club Barcelona and the Spanish national team."

article
# BERT = factsumm.calculate_bert_score(article, summary)


"Lionel Andrés Messi (born 24 June 1987) is an Argentine professional footballer who plays as a forward and captains both Spanish club Barcelona and the Argentina national team. Often considered as the best player in the world and widely regarded as one of the greatest players of all time, Messi has won a record six Ballon d'Or awards, a record six European Golden Shoes, and in 2020 was named to the Ballon d'Or Dream Team."

In [34]:
from metric.summac.summac.model_summac import SummaCZS, SummaCConv

article = document_texts
summary_pred_text = [decoded_sequence] if isinstance(decoded_sequence, str) else decoded_sequence

model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device=device, start_file="default", agg="mean")
SUMMAC = model_conv.score(article, summary_pred_text,  device=device)
torch.cuda.empty_cache()

<All keys matched successfully>


In [5]:
for i in eval_dataset:
    print(i)

{'document': 'Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.\nShe said there "probably was an element" of Labour\'s Remain campaign starting too late.\n"A lot of us were exhausted", she told BBC Radio Wales.\nSenior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.\nPlaid leader no longer ruling out coalition with Labour\nAMs to have say on Brexit deal, says Welsh Tory leader\nCorbyn\'s Labour set for heavy losses, says Stephen Kinnock\nAsked if Labour\'s Remain campaign started too late, she told the Eye on Wales programme: "I think there probably was an element of that.\n"And don\'t forget in Wales we had the assembly elections where people were not talking about the European Union. We were focused absolutely on those assembly elections."\nShe added: "I think it was a mistake not to be talking about the European

INFO:tensorflow:Reading checkpoint metric/bleurt/bleurt/BLEURT-20-D3.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20-D3
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20-D3
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: metric/bleurt/bleurt/BLEURT-20-D3/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2023-10-03 15:28:08.482056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-03 15:28:08.484762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-03 15:28:08.485737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

ResourceExhaustedError: Graph execution error:

Detected at node save/RestoreV2/_46 defined at (most recent call last):
<stack traces unavailable>
Detected at node save/RestoreV2/_46 defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;1c54a7adb7fc1780;/job:localhost/replica:0/task:0/device:GPU:0;edge_51_save/RestoreV2;0:0
	 [[{{node save/RestoreV2/_46}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[save/RestoreV2/_35]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED:  SameWorkerRecvDone u

In [6]:
# from metric.metrics.factsumm import FactSumm
# empty memory
import torch
torch.cuda.empty_cache()

from metric.metrics import *

factsumm = FactSumm()

article = "Lionel Andrés Messi (born 20 June 1987) is 'an Argentine' professional footballer who plays as a forward and captains both Spanish club Barcelona and the Argentina national team. Often considered as the best player in the world and widely regarded as one of the greatest players of all time, Messi has won a record six Ballon d'Or awards, a record six European Golden Shoes, and in 2020 was named to the Ballon d'Or Dream Team."

summary = "Lionel Andrés Messi (born 24 Aug 1997) is an Spanish 'professional footballer' who plays as a \n forward and captains both Spanish club Barcelona and the Spanish national team."

print(len(article))
print(len(summary))


BERT = factsumm.calculate_bert_score(article, summary)

427
175


Loading BERTScore Pipeline...

BERTScore Score
Precision: 0.8346061110496521
Recall: 0.8563808798789978
F1: 0.8460270762443542

In [4]:
torch.cuda.empty_cache()
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device=device, start_file="default", agg="mean")
SUMMAC = model_conv.score([article], [summary])['scores']
print(SUMMAC)

<All keys matched successfully>
[0.19466209411621094]


In [17]:
article = """Lionel Andrés Messi (born), Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.
She said there "probably was an element" of Labour's Remain campaign starting too late.
"A lot of us were exhausted", she told BBC Radio Wales.
Senior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.
Plaid leader no longer ruling out coalition with Labour
AMs to have say on Brexit deal, says Welsh Tory leader
Corbyn's Labour set for heavy losses, says Stephen Kinnock
Asked if Labour's Remain campaign started too late, she told the Eye on Wales programme: "I think there probably was an element of that.
mary_texts.
"""
article

"Lionel Andrés Messi (born), Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.\nShe said there 'probably was an element' of Labour's Remain campaign starting too late.\n'A lot of us were exhausted', she told BBC Radio Wales.\nSenior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.\nPlaid leader no longer ruling out coalition with Labour\nAMs to have say on Brexit deal, says Welsh Tory leader\nCorbyn's Labour set for heavy losses, says Stephen Kinnock\nAsked if Labour's Remain campaign started too late, she told the Eye on Wales programme: 'I think there probably was an element of that.\nmary_texts.\n"

In [1]:
# # calculate Bert scores using 
# import torch

# # remove cache
# torch.cuda.empty_cache()

# from evaluate import load
# bertscore = load("bertscore")

# # article = "Lionel Andrés Messi (born 20 June 1987) is an Argentine professional footballer who plays as a forward and captains both Spanish club Barcelona and the Argentina national team. Often considered as the best player in the world and widely regarded as one of the greatest players of all time, Messi has won a record six Ballon d'Or awards, a record six European Golden Shoes, and in 2020 was named to the Ballon d'Or Dream Team."
# article = """Baroness Morgan said Labour is in a serious situation if it cannot reconnect with working class communities.
# She said there "probably was an element" of Labour's Remain campaign starting too late.
# "A lot of us were exhausted", she told BBC Radio Wales.
# Senior Welsh Labour sources have privately said they were wary of talking about Brexit before the assembly election in case it drove Eurosceptic supporters towards UKIP.
# Plaid leader no longer ruling out coalition with Labour
# AMs to have say on Brexit deal, says Welsh Tory leader.
# """
# # article = article.strip('\n')
# summary = "Lionel Andrés Messi (born 24 Aug 1997) is an Spanish professional footballer who plays as a forward and captains both Spanish club Barcelona and the Spanish national team."

# # predictions = ["hello there", "general kenobi"]
# # references = ["hello there", "general kenobi"]
# predictions = [article]
# references = [summary]

# results = bertscore.compute(predictions=predictions, references=references, lang="en")
# print(results)

/home/erencan/anaconda3/envs/DL4NLP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-03 12:15:51.441518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-03 12:15:51.441551: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-03 12:15:51.441625: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-03 12:15:51.450249: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tensor

{'precision': [0.8000433444976807], 'recall': [0.7590187788009644], 'f1': [0.7789913415908813], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.33.2)'}


In [2]:
# # calculate Bert scores using 
import torch

# # remove cache
torch.cuda.empty_cache()

from evaluate import load

# predictions = ["hello there", "general kenobi"]
# references = ["hello there", "general kenobi"]
# bleurt = load("bleurt-large-512", module_type="metric")
# results = bleurt.compute(predictions=predictions, references=references)

FileNotFoundError: Couldn't find a module script at /home/erencan/UvA/MscYear2/DL4NLP/dl4nlp-text-summarization/bleurt-large-512/bleurt-large-512.py. Module 'bleurt-large-512' doesn't exist on the Hugging Face Hub either.